In [1]:
import pandas as pd
import random
import numpy as np
import torch
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(7)

In [2]:
numerical_data = pd.read_csv('all_after_preprocessingStopwords.csv')

In [3]:
len(numerical_data)

10251

In [4]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from transformers import BertTokenizer,BertModel,get_linear_schedule_with_warmup, RobertaTokenizer, RobertaModel
from torch.utils.data import Dataset, DataLoader
def Data_preprocessing(df):
    scaler = MinMaxScaler()
    
    # Numerical Feature
    numerical_features = ["bedroom","bedroomAboveGrade","bedroomBelowGrade","bathroom", "bathroomTotal","bathroomPartial", 
                        "totalParkingSpaces", "storeys", "maintenanceFees",  'landSize']
    
    X_num = df[numerical_features]
    X_num = scaler.fit_transform(X_num)
    X_num = pd.DataFrame(X_num)
    df[numerical_features] = X_num
    df["longitude"] = df["longitude"] *0.01
    df["latitude"] = df["latitude"] *0.01
    numerical_features += ['latitude', 'longitude']
    # Boolean Feature
    boolean_features=['parkingAttachedGarage',
       'parkingUnderground', 'parkingInsideEntry', 'parkingSurfaced',
       'parkingOversize', 'parkingGravel', 'parkingGarage', 'parkingShared',
       'parkingDetachedGarage', 'parkingCarport', 'parkingInterlocked',
       'parkingVisitorParking','amenityClubhouse', 'amenityCarWash', 'amenityMusicRoom',
       'amenityStorageLocker', 'amenitySauna', 'amenityPartyRoom',
       'amenityRecreationCentre', 'amenityGuestSuite', 'amenityFurnished',
       'amenityLaundryFacility', 'amenityExerciseCentre',
       'amenityLaundryInSuite', 'amenitySecurity', 'amenityWhirlpool',
       'efinishWood', 'efinishBrick', 'efinishHardboard', 'efinishWoodsiding',
       'efinishLog', 'efinishMetal', 'efinishSteel', 'efinishStone',
       'efinishWoodshingles', 'efinishStucco', 'efinishSiding',
       'efinishConcrete', 'efinishShingles', 'efinishAluminumsiding',
       'efinishCedarshingles', 'efinishVinyl', 'efinishVinylsiding',
       'featurePetNotAllowed', 'AirportNearby',
       'GolfNearby', 'MarinaNearby', 'ShoppingNearby', 'WaterNearby',
       'WorshipPlaceNearby', 'RecreationNearby', 'PlaygroundNearby',
       'PublicTransitNearby', 'ParkNearby', 'SchoolsNearby', 'HospitalNearby',
       'HighwayNearby', 'SkiAreaNearby']
    
    # Category Feature
    cate_features = ['city', 'typeBuilding', 'title', 'styleAttach', 
                   'cooling',  'basementType', 'basementFinish',
                   'heatingType1', 'heatingType2', 'heatingEnergy1', 'heatingEnergy2', 
                   'featureLotSlope', 'featureDriveway', 'featureLotPositionType',
       'featureOutdoorAreaType', 'featureOutdoorLandscape',
       'featureAdditionalFacility']
    
    X_category=df[cate_features]
    for col in cate_features:
        X_category[col] = X_category[col].astype('category')
        X_category[col] = X_category[col].cat.codes
    df[cate_features] = X_category
    
    # Label Price
    price_range = []
    
    for price in df["price"]:
        if price < 5e5:
            price_range.append(0)
        elif 5e5<=price < 15e5:
            price_range.append(1)
        elif 15e5<=price < 25e5:
            price_range.append(2)
        elif 25e5<=price < 35e5:
            price_range.append(3)
        elif 35e5<=price < 80e5:
            price_range.append(4)
        else:
            price_range.append(5)
            
#     df = df.reset_index(drop=True)
    df['price_range'] = price_range
    df = df.dropna()
    return df, boolean_features, cate_features, numerical_features

In [5]:
from sklearn.model_selection import train_test_split
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
preprocessed_data, boolean_features, cate_features, numerical_features = Data_preprocessing(numerical_data)

<ipython-input-4-d7c27bed41cf>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_category[col] = X_category[col].astype('category')
<ipython-input-4-d7c27bed41cf>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_category[col] = X_category[col].cat.codes


In [6]:
cate_num = len(cate_features)
bool_num =  len(boolean_features)
num_num = len(numerical_features)
print('The number of categorical feature is : {}, boolean feature is : {}, numerical feature is : {}'.format(cate_num, bool_num, num_num))

The number of categorical feature is : 17, boolean feature is : 58, numerical feature is : 12


In [7]:
class Generate_dataset(Dataset):
    def __init__(self, data_df, tokenizer, boolean_features,cate_features, numerical_features):
        self.MAX_SEQ_LEN = 128
        self.data = data_df
        self.tokenizer = tokenizer
        self.boolean = boolean_features
        self.cate = cate_features
        self.numerical = numerical_features

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx): 
        if torch.is_tensor(idx):
            idx = idx.tolist()

        boolean_feature = torch.tensor(list(self.data.iloc[idx][self.boolean]))
        cate_feature = torch.tensor(list(self.data.iloc[idx][self.cate]))
        numerical_feature = torch.tensor(list(self.data.iloc[idx][self.numerical]))
        description = self.data.iloc[idx]['description']
        word_encode = torch.tensor(self.tokenizer.encode(text= description,max_length=self.MAX_SEQ_LEN,padding='max_length', truncation=True))
#         label = torch.tensor(float(self.data.iloc[idx]['price']))
        label_range =  torch.tensor(int(self.data.iloc[idx]['price_range']))
        
        return {'description': word_encode, 'numerical_feature':numerical_feature, 'cate_feature':cate_feature, 'boolean_feature':boolean_feature, 'label_range':label_range}

In [8]:
num_feature = cate_features + numerical_features + boolean_features
res_feature = ['description', 'price_range']

In [9]:
final_data = preprocessed_data[num_feature + res_feature]

In [10]:
X_train, X_test = train_test_split(final_data, test_size=0.1, random_state=13) 
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [11]:
X_train.groupby(['price_range']).size()

price_range
0     994
1    6397
2    1058
3     389
4     297
5      83
dtype: int64

In [12]:
from imblearn.over_sampling import SMOTE 
strategy = {3:889, 4:797, 5:583}
sm = SMOTE(random_state=42,sampling_strategy=strategy)
X_res, y_res = sm.fit_resample(X_train[num_feature], X_train['price_range'])

In [13]:
X_res['price_range'] = y_res

In [14]:
X_res.groupby(['price_range']).size()

price_range
0     994
1    6397
2    1058
3     889
4     797
5     583
dtype: int64

In [15]:
def import_sentence(df, df_smote, path):
    label_3_sentences = []
    label_4_sentences = []
    label_5_sentences = []
    with open(path+'label_3_sentence.txt', 'r') as f:
        lines = f.readlines()
        for line in lines:
            label_3_sentences.append(line.strip())
    with open(path+'label_4_sentence.txt', 'r') as f:
        lines = f.readlines()
        for line in lines:
            label_4_sentences.append(line.strip())            
    with open(path+'label_5_sentence.txt', 'r') as f:
        lines = f.readlines()
        for line in lines:
            label_5_sentences.append(line.strip())
            
    org_sentences = list(df['description'])
    new_sentences = []
    idx3, idx4, idx5 = 0,0,0
    
    for i in range(len(df), len(df_smote)):

        if df_smote.iloc[i]['price_range'] == 3:
            new_sentences.append(label_3_sentences[idx3])
            idx3+=1
        elif df_smote.iloc[i]['price_range'] == 4:
            new_sentences.append(label_4_sentences[idx4])
            idx4+=1
        elif df_smote.iloc[i]['price_range'] == 5:
            new_sentences.append(label_5_sentences[idx5])
            idx5+=1
    
    df_smote['description'] = org_sentences + new_sentences
    return df_smote

In [16]:
df_smote = import_sentence(X_train, X_res, 'generated_sentence/')

In [17]:
Train_dataset = Generate_dataset(df_smote, tokenizer, boolean_features,cate_features, numerical_features)
Test_dataset = Generate_dataset(X_test, tokenizer, boolean_features,cate_features, numerical_features)

In [18]:
Train_dataloader = DataLoader(Train_dataset, batch_size = 16,shuffle=True)
Test_dataloader = DataLoader(Test_dataset, batch_size = 16)

In [19]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    device = torch.device('cpu')
BERT_org = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
import math
class BERT_Predictor(torch.nn.Module):
    def __init__(self, pretrained_model, tokenizer, predictor_size = 1, multi_mode = None):
        super(BERT_Predictor, self).__init__()
        
        self.bert = pretrained_model
        self.PAD_INDEX = tokenizer.pad_token_id
    
        self.final_embed = 36
        
        if multi_mode is not None:
            self.cat_num = multi_mode['cate_num']
            self.bool_num = multi_mode['bool_num']
            self.num_num = multi_mode['num_num']
        
        self.cat_encoder = torch.nn.Linear(self.cat_num,self.final_embed)
        self.bool_encoder = torch.nn.Linear(self.bool_num,self.final_embed)
        self.num_encoder = torch.nn.Linear(self.num_num,self.final_embed)
        self.des_encoder =  torch.nn.Linear(768,self.final_embed)
        
        self.multihead_attn = torch.nn.MultiheadAttention(self.final_embed, 12, dropout = 0.2, batch_first = True)
        self.layer_norm = torch.nn.LayerNorm(self.final_embed)
        self.dropout = torch.nn.Dropout(0.2)
        
        self.predictor = torch.nn.ModuleList()
        for _ in range(predictor_size):
            self.predictor.append(torch.nn.Linear(self.final_embed,self.final_embed))
            self.predictor.append(torch.nn.ReLU())
            self.predictor.append(torch.nn.Dropout(0.2))
            
        self.prediction = torch.nn.Linear(self.final_embed, 6)


    def forward(self,input_ids, cat_feature = None, numerical_feature = None, bool_feature = None, check_attention = False):
        attention_mask = (input_ids != self.PAD_INDEX).type(torch.uint8)

        outputs = self.bert(input_ids,attention_mask=attention_mask)
        final_output = outputs[0]
        
        cat_feature = cat_feature.float()
        numerical_feature = numerical_feature.float()
        bool_feature = bool_feature.float()
        
        cat_feature_final = self.cat_encoder(cat_feature).unsqueeze(1)
        numerical_feature_final = self.num_encoder(numerical_feature).unsqueeze(1)
        bool_feature_final = self.bool_encoder(bool_feature).unsqueeze(1)
        des_output = self.des_encoder(final_output)
        
        numerical = torch.cat((cat_feature_final, numerical_feature_final, bool_feature_final, des_output), 1)
        
        
        attn_output, attention = self.multihead_attn(query = numerical, key = numerical, value = numerical, need_weights=True)
        

        attn_output  = self.dropout(attn_output)
        attn_output = attn_output
        attn_output = self.layer_norm(attn_output + numerical)
        
        pooled_output = torch.mean(attn_output, dim = 1)
        
        
        for layer in self.predictor:
            pooled_output = layer(pooled_output)

        prediction = self.prediction(pooled_output)
        
        if check_attention:
            return attention
        
        return prediction

In [21]:
multi_mode = {'cate_num': cate_num, 'bool_num':bool_num, 'num_num':num_num}
Bert_regressor = BERT_Predictor(BERT_org, tokenizer,multi_mode = multi_mode).to(device)

In [22]:
from tqdm import tqdm
def train(model,optimizer, train_iter,device,num_epochs):
    model.train()
    R2_final = []
    MSE = []
    R2_val = 0
    MSE_val = 0
    step = 0
    for _ in range(num_epochs):
        for data in tqdm(train_iter):
            optimizer.zero_grad()
            source = data['description'].to(device)
            target_range = data['label_range'].to(device).unsqueeze(-1)
            cat_feature = data['cate_feature'].to(device)
            numerical_feature = data[ 'numerical_feature'].to(device)
            bool_feature = data['boolean_feature'].to(device)
            cls = model(input_ids=source,cat_feature = cat_feature, numerical_feature = numerical_feature, bool_feature = bool_feature)
            loss = torch.nn.CrossEntropyLoss()(cls, target_range.long().squeeze())
            loss.backward()
            optimizer.step()
            scheduler.step()
        print('CLS loss is:{}'.format(float(loss)))

In [23]:
num_epochs = 5
lr_rate = 3e-5

In [24]:
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
param_optimizer = list(Bert_regressor.named_parameters())
optimizer_grouped_parameters = [
        {
                'params':[p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                'weight_decay':0.01
        },
        {
                'params':[p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                'weight_decay':0.0
        }
]
total_steps = len(Train_dataloader) * num_epochs
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, optimizer_grouped_parameters[0]['params']), lr= lr_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0.1*total_steps,num_training_steps=total_steps)

In [25]:
# train(Bert_regressor,optimizer, Train_dataloader,device,num_epochs)

In [26]:
# torch.save(Bert_regressor.state_dict(), 'Bert_self_attention1.pt')

In [27]:
Bert_regressor.load_state_dict(torch.load('Bert_self_attention1.pt'))

<All keys matched successfully>

In [28]:
def evaluate_attacker(model, test_iter):
    fin_targets_range = []
    fin_outputs_range = []
    attention_matrix = []
    with torch.no_grad():
        model.eval()
        for data in tqdm(test_iter):
            source = data['description'].to(device)
            target_range = data['label_range'].to(device).unsqueeze(-1)
            cat_feature = data['cate_feature'].to(device)
            numerical_feature = data[ 'numerical_feature'].to(device)
            bool_feature = data['boolean_feature'].to(device)
            att = model(input_ids=source,cat_feature = cat_feature, numerical_feature = numerical_feature, bool_feature = bool_feature,check_attention = True)
            attention_matrix.append(att.tolist())
#             try:
            fin_targets_range.extend(target_range.tolist())
#                 fin_outputs_range.extend(torch.argmax(cls,1).tolist())
#             except:
#                 print('---')
    return attention_matrix,fin_targets_range

In [29]:
attention_matrix,fin_targets_range = evaluate_attacker(Bert_regressor, Test_dataloader)

  0%|                                                                                           | 0/65 [00:00<?, ?it/s]<ipython-input-7-4d6e1eb4bd00>:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  boolean_feature = torch.tensor(list(self.data.iloc[idx][self.boolean]))
100%|██████████████████████████████████████████████████████████████████████████████████| 65/65 [00:10<00:00,  6.13it/s]


In [30]:
fin_targets = np.array(fin_targets_range).reshape(-1)

In [36]:
class_1_idx = [idx for idx, i in enumerate(fin_targets) if i == 0]
class_2_idx = [idx for idx, i in enumerate(fin_targets) if i == 1]
class_3_idx = [idx for idx, i in enumerate(fin_targets) if i == 2]
class_4_idx = [idx for idx, i in enumerate(fin_targets) if i == 3]
class_5_idx = [idx for idx, i in enumerate(fin_targets) if i == 4]
class_6_idx = [idx for idx, i in enumerate(fin_targets) if i == 5]

In [37]:
org = torch.tensor(attention_matrix[:-1]).view(-1, 131, 131)
res = torch.tensor(attention_matrix[-1])
final = torch.cat((org, res), dim = 0)
uniform_data1 = torch.mean(torch.index_select(final, 0, torch.tensor(class_1_idx)), dim = 0).numpy()[:4,:4]
uniform_data2 = torch.mean(torch.index_select(final, 0, torch.tensor(class_2_idx)), dim = 0).numpy()[:4,:4]
uniform_data3 = torch.mean(torch.index_select(final, 0, torch.tensor(class_3_idx)), dim = 0).numpy()[:4,:4]
uniform_data4 = torch.mean(torch.index_select(final, 0, torch.tensor(class_4_idx)), dim = 0).numpy()[:4,:4]
uniform_data5 = torch.mean(torch.index_select(final, 0, torch.tensor(class_5_idx)), dim = 0).numpy()[:4,:4]
uniform_data6 = torch.mean(torch.index_select(final, 0, torch.tensor(class_6_idx)), dim = 0).numpy()[:4,:4]

In [38]:
import matplotlib.pylab as plt
import random
import matplotlib.gridspec as gridspec
import seaborn as sns

x_axis_labels = ['Cat.', 'Num.', 'Bool.', 'Text'] 
y_axis_labels = ['Cat.', 'Num.', 'Bool.', 'Text']

sns.set(font_scale=2)
gs = gridspec.GridSpec(2,3)
fig = plt.figure(figsize=(30,12), dpi=600)
#first plot
ax = fig.add_subplot(gs[0])

matrix = np.triu(uniform_data1)
matrix[0][0] = 0
matrix[1][1] = 0
matrix[2][2] = 0
matrix[3][3] = 0

sns.heatmap(uniform_data1, linewidth=0.5,annot=True,xticklabels=x_axis_labels, yticklabels=y_axis_labels, mask=matrix)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.tick_params(labelsize=18)
ax.set_title('Class 1',fontsize=20,fontweight='bold')

ax = fig.add_subplot(gs[1], sharex=ax)
matrix = np.triu(uniform_data2)
matrix[0][0] = 0
matrix[1][1] = 0
matrix[2][2] = 0
matrix[3][3] = 0

sns.heatmap(uniform_data2, linewidth=0.5,annot=True,xticklabels=x_axis_labels, yticklabels=y_axis_labels,mask=matrix)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.tick_params(labelsize=18)
ax.set_title('Class 2',fontsize=20,fontweight='bold')

ax = fig.add_subplot(gs[2], sharex=ax)
matrix = np.triu(uniform_data3)
matrix[0][0] = 0
matrix[1][1] = 0
matrix[2][2] = 0
matrix[3][3] = 0
sns.heatmap(uniform_data3, linewidth=0.5,annot=True,xticklabels=x_axis_labels, yticklabels=y_axis_labels,mask=matrix)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.tick_params(labelsize=18)
ax.set_title('Class 3',fontsize=20,fontweight='bold')
# ax.legend(fontsize=18)

#forth plot
ax = fig.add_subplot(gs[3], sharex=ax)
matrix = np.triu(uniform_data4)
matrix[0][0] = 0
matrix[1][1] = 0
matrix[2][2] = 0
matrix[3][3] = 0
sns.heatmap(uniform_data4, linewidth=0.5,annot=True,xticklabels=x_axis_labels, yticklabels=y_axis_labels,mask=matrix)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.tick_params(labelsize=18)
ax.set_ylabel('TRP',fontsize=20,loc="center")
ax.yaxis.set_label_position("right")
ax.set_title('Class 4',fontsize=20,fontweight='bold')

ax = fig.add_subplot(gs[4], sharex=ax)
matrix = np.triu(uniform_data5)
matrix[0][0] = 0
matrix[1][1] = 0
matrix[2][2] = 0
matrix[3][3] = 0
sns.heatmap(uniform_data5, linewidth=0.5,annot=True,xticklabels=x_axis_labels, yticklabels=y_axis_labels,mask=matrix)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.set_title('Class 5',fontsize=20,fontweight='bold')
ax.tick_params(labelsize=18)

ax = fig.add_subplot(gs[5], sharex=ax)
matrix = np.triu(uniform_data6)
matrix[0][0] = 0
matrix[1][1] = 0
matrix[2][2] = 0
matrix[3][3] = 0
sns.heatmap(uniform_data6, linewidth=0.5,annot=True,xticklabels=x_axis_labels, yticklabels=y_axis_labels,mask=matrix)
sns.set(font_scale=0.1)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.set_title('Class 6',fontsize=20,fontweight='bold')
ax.tick_params(labelsize=18)

In [34]:
fig.savefig("attention_graph.pdf",bbox_inches='tight',dpi=fig.dpi,pad_inches=0.0)